# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [2]:
# imports
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from dotenv import load_dotenv
import os

# constantes

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


# Configura el entorno
# Cargamos las variables del fichero .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

system_prompt="Eres un experto programador en Python 3.12 y vas a responderme todas las dudas que tenga con respcecto \
a fragmentos de código que te de, así como explicarme el código en cuestión. \
tambien me explicaras terminos que no entienda de la programación"

def obtain_user_prompt():
    user_question = [] 
    line = input("Introduce un prompt que termine con una línea vacía: ")
    while line:
        user_question.append(line)
        line = input() 

    user_question = "\n".join(user_question)
    return user_question


# Haz que gpt-4o-mini responda, con streaming
def stream_response():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": obtain_user_prompt()}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [3]:
stream_response()

Introduce un prompt que termine con una línea vacía:  Explica qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 


Claro, vamos a descomponer y explicar el fragmento de código que has proporcionado:

python
yield from {book.get("author") for book in books if book.get("author")}


### Desglose del código

1. **Comprensión de conjunto (`set comprehension`)**:
   - `{book.get("author") for book in books if book.get("author")}` es una comprensión de conjunto que itera sobre una colección llamada `books`.
   - Aquí, `book` representa cada elemento (que se espera que sea un diccionario u objeto similar) en la colección `books`.
   - `book.get("author")` obtiene el valor asociado a la clave `"author"` de cada `book`. Si la clave `"author"` no existe, `get` devolverá `None` en lugar de generar una excepción o error.
   - `if book.get("author")` actúa como un filtro que asegura que sólo los `authors` que existen y no son `None` se incluyan en el conjunto resultante.

2. **Uso de `yield from`**:
   - `yield from` es una forma de delegar la generación de valores a otro generador o iterable. En este caso, el conjunto generado por la comprensión (`{...}`) se está pasando a `yield from`.
   - Esto significa que definimos un generador que produce cada uno de los elementos del conjunto resultante de autores (únicos) extraídos de los libros.

### Resumen de lo que hace el código

- El código primero crea un conjunto de nombres de autores (`{author1, author2, ...}`) a partir de la lista `books`.
- Solo se incluyen aquellos autores que están presentes (es decir, que no son `None`).
- Luego, se usa `yield from` para devolver uno por uno cada uno de estos autores como parte de un generador.

### ¿Por qué se usa este código?

- **Eliminación de duplicados**: Usar un conjunto (`set`) automáticamente elimina cualquier autor duplicado, lo que significa que cada autor solo aparecerá una vez en el resultado.
- **Generador eficiente**: `yield from` permite producir los resultados uno a uno, lo que es eficiente en términos de memoria, especialmente si la lista de `books` es grande.

### Ejemplo de uso

Supongamos que tenemos el siguiente código en un generador:

python
def generate_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Ejemplo de uso
books = [
    {"author": "Author A"},
    {"author": "Author B"},
    {"author": "Author A"},
    {"author": None},
    {"title": "Book C"},
]

for author in generate_authors(books):
    print(author)


**Salida del ejemplo**:

Author B
Author A


Como puedes ver, `Author A` solo aparece una vez en la salida, aunque estaba presente dos veces en la lista `books`. El `None` y el libro sin autor son ignorados.


Si tienes alguna pregunta adicional, no dudes en preguntar.

In [4]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# imports
from IPython.display import Markdown, display, update_display
import requests

# constantes

MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}


def obtain_user_prompt():
    user_question = [] 
    line = input("Introduce un prompt que termine con una línea vacía: ")
    while line:
        user_question.append(line)
        line = input() 

    user_question = "\n".join(user_question)
    return user_question


# Haz que Llama 3.2 responda
def llama_tutor():

    messages=[{"role": "user", "content": obtain_user_prompt()}]
    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": False
    }
    
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    print(response.json()['message']['content'])

In [6]:
llama_tutor()

Introduce un prompt que termine con una línea vacía:  Explica qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 


Este código utiliza un concepto llamado "generador" en Python, que es una función especializada para producir un valor de manera iterativa. En este caso, el código está utilizando la cláusula `yield from` y la función `get()`.

**Estructura**

```python
yield from {book.get("author") for book in books if book.get("author")}
```

**Componentes**

1. `{book.get("author") for book in books if book.get("author")}`: Esta es una expresión de generación, que utiliza un patron `for` para iterar sobre los elementos del conjunto (`books`). La condición `if book.get("author")` filtra los elementos que no tienen el atributo `"author"` asociado.
2. `yield from`: Esta palabra clave indica que la función es un generador. Cuando se llama a una función con `yield from`, su comportamiento se cambia para que produzca valores de manera iterativa.

**Cómo funciona**

Cuando se ejecuta este código, aquí está lo que sucede:

1. La expresión `{book.get("author") for book in books if book.get("author")}` crea 